## Agglomerative clustering with categorical data


In [ ]:
# Zoo dataset : https://www.kaggle.com/datasets/uciml/zoo-animal-classification/code?datasetId=586&searchQuery=hier&select=zoo.csv

import pandas as pd
import numpy as np

import seaborn as sns
from matplotlib import pyplot as plt

from scipy.cluster.hierarchy import dendrogram, linkage, fcluster

from ch_9_funcs import plot_cluster_dendrogram, cophenetic_corr

sns.set_theme()

## Animal zoo dataset

The Zoo Dataset is a comprehensive collection of data about various animals found in zoos worldwide. The dataset is composed of several attributes related to these animals, such as:

- `animal_name`: The name of the animal.

- `hair`: Indicates if the animal has hair (1 for yes, 0 for no).
- `feathers`: Indicates if the animal has feathers (1 for yes, 0 for no).
- `eggs`: Indicates if the animal lays eggs (1 for yes, 0 for no).
- `milk`: Indicates if the animal produces milk (1 for yes, 0 for no).
- `airborne`: Indicates if the animal can fly (1 for yes, 0 for no).
- `aquatic`: Indicates if the animal lives in water (1 for yes, 0 for no).
- `predator`: Indicates if the animal is a predator (1 for yes, 0 for no).
- `toothed`: Indicates if the animal has teeth (1 for yes, 0 for no).
- `backbone`: Indicates if the animal has a backbone (1 for yes, 0 for no).
- `breathes`: Indicates if the animal breathes air (1 for yes, 0 for no).
- `venomous`: Indicates if the animal is venomous (1 for yes, 0 for no).
- `fins`: Indicates if the animal has fins (1 for yes, 0 for no).
- `legs`: Number of legs the animal has (integer value).
- `tail`: Indicates if the animal has a tail (1 for yes, 0 for no).
- `domestic`: Indicates if the animal is domesticated (1 for yes, 0 for no).
- `catsize`: Indicates if the animal is cat-sized or larger (1 for yes, 0 for no).
- `class_type`: Numerical code indicating the animal's taxonomic class.


Citation : Forsyth,Richard. (1990). Zoo. UCI Machine Learning Repository. https://doi.org/10.24432/C5R59V.

In [ ]:
# Load the data
zoo_path = 'data/hierarchical/zoo/zoo.csv'
zoo_df = pd.read_csv(zoo_path)

# Load class mapping
class_mappings = 'data/hierarchical/zoo/class.csv'
class_mapper = pd.read_csv(class_mappings)

In [ ]:
zoo_df.head()

In [ ]:
class_mapper

In [ ]:
zoo_df.shape

In [ ]:
zoo_df.isna().sum(axis=0)

In [ ]:
# Check number of unique animals 
zoo_df['animal_name'].nunique()

In [ ]:
# Drop duplicates since there should be one species per row
zoo_df = zoo_df.drop_duplicates('animal_name')
zoo_df.shape

In [ ]:
# Map class id to class name
class_map_dict = class_mapper.set_index('Class_Number')['Class_Type'].to_dict()
zoo_df['class_type'] = zoo_df['class_type'].map(class_map_dict)

# Extract class as separate object and drop class from zoo_df
animal_class = zoo_df['class_type'] 
zoo_df = zoo_df.drop(columns=['class_type'])

In [ ]:
# Set animal name as index
zoo_df = zoo_df.set_index('animal_name', drop=True)

In [ ]:
_ = zoo_df.hist(figsize=(7,7))
plt.tight_layout()

### Agglomerative clustering with hamming distance

In [ ]:
linkage_matrix = linkage(
    zoo_df.to_numpy(), 
    method='average', 
    metric='hamming'
)

In [ ]:
cophenetic_corr(
    linkage_matrix=linkage_matrix,
    data_mtx = zoo_df.to_numpy(),
    distance_measure='hamming'
)

In [ ]:
linkage_matrix[:,2] += 0.001

In [ ]:
plt.rcParams['figure.figsize'] = [12, 24]
_ = dendrogram(
    linkage_matrix, 
    labels=zoo_df.index,
    orientation='left',
    leaf_font_size=8
)

In [ ]:
plt.rcParams['figure.figsize'] = [12, 24]
_ = dendrogram(
    linkage_matrix, 
    labels=animal_class.to_numpy(),
    orientation='left',
    leaf_font_size=8
)

In [ ]:
# Perform clustering 
clusters = fcluster(Z=linkage_matrix, t=0.28, criterion='distance')
clusters

In [ ]:
_ = plot_cluster_dendrogram(
    linkage_matrix=linkage_matrix,
    dataset_df=zoo_df,
    clusters=clusters,
    leaf_font_size=7,
    labels=animal_class.to_numpy()
)

In [ ]:
_ = plot_cluster_dendrogram(
    linkage_matrix=linkage_matrix,
    dataset_df=zoo_df,
    clusters=clusters,
    leaf_font_size=7
)